In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/WSTM-Proj

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1oWYt7oVJSbEFyIJzfizy8NTc6cl6XI09/WSTM-Proj


## Foursquare Files

In [1]:
# change small to False for Foursquare or to True for Foursquare_small
dname = 'Foursquare'
small = True
region = True
# min_period = dt.datetime(2013, 11, 1, 0, 0, 0)
# max_period = dt.datetime(2014, 1, 31, 23, 59, 59)

# # 5 states - east coast
# min_lat = 37.431600
# min_long = -86.134900
# max_lat = 41.203300
# max_long = -77.194500

# # west coast
# min_lat = 34.048900
# min_long = -120.554200
# max_lat = 47.751100
# max_long = -105.782100

# Entire US
min_lat = 25.700000
min_long = -124.440000
max_lat = 49.230000
max_long = -66.570000

# print(min_period, max_period)
print(min_lat, max_lat, min_long, max_long)

25.7 49.23 -124.44 -66.57


In [2]:
max_long > min_long

True

### Main Generation

In [4]:
df = pd.read_csv('data/Foursquare/dataset_WWW2019/dataset_WWW_Checkins_anonymized.txt', sep='\t', header=None)
df.columns = ['user','locid','utc','time']
print(df.shape)
# (22809624, 4)
df.head()

(22809624, 4)


,user,locid,utc,time
0,822121,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180
1,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300
2,113817,4b4bade2f964a520cfa326e3,Tue Apr 03 18:00:09 +0000 2012,-240
3,14732,4c143cada5eb76b0dc7dc1b7,Tue Apr 03 18:00:09 +0000 2012,-240
4,1397630,4e88cf4ed22d53877981fdab,Tue Apr 03 18:00:09 +0000 2012,-300


In [5]:
if region:
  df_poi = pd.read_csv('data/Foursquare/dataset_WWW2019/raw_POIs.txt', sep='\t', header=None)
  df_poi.columns = ['locid','lat','long','cat','country']
  df_poi = df_poi[['locid','lat','long']]
  print(df_poi.shape)
  df_poi = df_poi[((df_poi.lat >= min_lat) & (df_poi.lat <= max_lat))]
  df_poi = df_poi[((df_poi.long >= min_long) & (df_poi.long <= max_long))]
  print(df_poi.shape)
  reg_list = list(set(df_poi.locid))
  del df_poi
  # df_poi.head()

(11180160, 3)
(2083109, 3)


In [6]:
# Get date in datetime format
df['utc_clipped'] = df.utc.str.replace(" +0000","", regex = False).str.slice(4) #50s
df = df[~df.utc_clipped.isin(['1239673244639234'])]
df['utc_dt'] = pd.to_datetime(df.utc_clipped, format = '%b %d %H:%M:%S %Y') 

# Filter dataset if small needed
# if small:
#   df = df[(df.utc_dt >= min_period)&(df.utc_dt <= max_period)]
if region:
  df = df[(df.locid.isin(reg_list))]

print(df.shape)
min_date = df.utc_dt.min() - dt.timedelta(minutes = 1)
print(min_date)

df.head(2)
# (22809624, 4)
# 2012-04-03 17:59:07

# ### SMALL
# (435639, 6)
# 2013-12-02 08:51:00

(3377448, 6)
2012-04-03 17:59:08


,user,locid,utc,time,utc_clipped,utc_dt
1,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,Apr 03 18:00:08 2012,2012-04-03 18:00:08
2,113817,4b4bade2f964a520cfa326e3,Tue Apr 03 18:00:09 +0000 2012,-240,Apr 03 18:00:09 2012,2012-04-03 18:00:09


In [7]:
# Filter users with more than 5 check-ins
users = df.groupby('user',as_index=False).agg({'locid':'count'})
df_selected = df[df.user.isin(users[users.locid > 5].user)]
print(df_selected.shape)
# del df

# All users have more than 5 check-ins

# ### SMALL
# (338971, 6)

(3366689, 6)


In [8]:
# Get locations with at least 10 check-ins
checkins = df_selected.groupby('locid', as_index= False).agg({'user':'count'})
checkins = checkins[checkins.user > 9]
print(checkins.shape)

# Introduce the numeric location ids
checkins = checkins.reset_index(drop = True).drop(columns = ['user']).reset_index()
checkins.columns = ['locid_num', 'locid']
checkins['locid_num'] = checkins.locid_num + 1
print(checkins.shape)
print(checkins.locid_num.min(), checkins.locid_num.max())

# Save the locid to locid_num mapping
# if small:
#   checkins.to_csv("Foursquare" + "/small_small_location_mapping.csv", index = False)
# else:
#   checkins.to_csv(dname + "/location_mapping.csv", index = False)
checkins.head(2)
# (409428, 2)

# ### SMALL
# (3109, 2)
# 1 3109
# 3631
# 3391

(66255, 2)
(66255, 2)
1 66255


,locid_num,locid
0,1,3fd66200f964a52000e71ee3
1,2,3fd66200f964a52000e81ee3


In [9]:
# Filter for locations with atleast 10 check-ins and add the numeric location ids
print(df_selected.shape)
df_selected = df_selected.merge(checkins, on=['locid'], how = 'inner')
print(df_selected.shape)

df_selected.head(1)
# (22809624, 4)
# (15397923, 5)

# ### SMALL
# (338971, 6)
# (58986, 7)

(3366689, 6)
(1871606, 7)


,user,locid,utc,time,utc_clipped,utc_dt,locid_num
0,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,Apr 03 18:00:08 2012,2012-04-03 18:00:08,22272


In [10]:
# Filter users with more than 2 check-ins
if region or small:
  users = df_selected.groupby('user',as_index = False).agg({'locid':'count'})
  df_selected = df_selected[df_selected.user.isin(users[users.locid > 2].user)]
  print(df_selected.shape)

  print(df_selected.locid_num.nunique())
  print(df_selected.locid_num.min(), df_selected.locid_num.max())

df_selected.head()

(1871034, 7)
66255
1 66255


,user,locid,utc,time,utc_clipped,utc_dt,locid_num
0,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,Apr 03 18:00:08 2012,2012-04-03 18:00:08,22272
1,208842,4b4606f2f964a520751426e3,Wed Apr 04 19:20:35 +0000 2012,-300,Apr 04 19:20:35 2012,2012-04-04 19:20:35,22272
2,208842,4b4606f2f964a520751426e3,Mon Apr 09 13:57:42 +0000 2012,-300,Apr 09 13:57:42 2012,2012-04-09 13:57:42,22272
3,208842,4b4606f2f964a520751426e3,Thu Apr 12 13:27:53 +0000 2012,-300,Apr 12 13:27:53 2012,2012-04-12 13:27:53,22272
4,208842,4b4606f2f964a520751426e3,Fri Apr 13 15:41:49 +0000 2012,-300,Apr 13 15:41:49 2012,2012-04-13 15:41:49,22272


In [11]:
# # Save Pandas df with date format for easy load later
# df_selected.to_csv("Foursquare/selected_check_ins.csv", index = False)

In [12]:
# converting date to time in minutes
df_selected['date_init'] = pd.to_datetime(min_date)
df_selected['time_in_min'] = (df_selected.utc_dt - df_selected.date_init).dt.total_seconds().div(60).astype(float)
df_selected.head()

,user,locid,utc,time,utc_clipped,utc_dt,locid_num,date_init,time_in_min
0,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,Apr 03 18:00:08 2012,2012-04-03 18:00:08,22272,2012-04-03 17:59:08,1.000000
1,208842,4b4606f2f964a520751426e3,Wed Apr 04 19:20:35 +0000 2012,-300,Apr 04 19:20:35 2012,2012-04-04 19:20:35,22272,2012-04-03 17:59:08,1521.450000
2,208842,4b4606f2f964a520751426e3,Mon Apr 09 13:57:42 +0000 2012,-300,Apr 09 13:57:42 2012,2012-04-09 13:57:42,22272,2012-04-03 17:59:08,8398.566667
3,208842,4b4606f2f964a520751426e3,Thu Apr 12 13:27:53 +0000 2012,-300,Apr 12 13:27:53 2012,2012-04-12 13:27:53,22272,2012-04-03 17:59:08,12688.750000
4,208842,4b4606f2f964a520751426e3,Fri Apr 13 15:41:49 +0000 2012,-300,Apr 13 15:41:49 2012,2012-04-13 15:41:49,22272,2012-04-03 17:59:08,14262.683333


In [13]:
# Round the time_in_min
df_selected['time_in_min'] = df_selected['time_in_min'].round(0).astype(int)
df_selected.head()

,user,locid,utc,time,utc_clipped,utc_dt,locid_num,date_init,time_in_min
0,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,Apr 03 18:00:08 2012,2012-04-03 18:00:08,22272,2012-04-03 17:59:08,1
1,208842,4b4606f2f964a520751426e3,Wed Apr 04 19:20:35 +0000 2012,-300,Apr 04 19:20:35 2012,2012-04-04 19:20:35,22272,2012-04-03 17:59:08,1521
2,208842,4b4606f2f964a520751426e3,Mon Apr 09 13:57:42 +0000 2012,-300,Apr 09 13:57:42 2012,2012-04-09 13:57:42,22272,2012-04-03 17:59:08,8399
3,208842,4b4606f2f964a520751426e3,Thu Apr 12 13:27:53 +0000 2012,-300,Apr 12 13:27:53 2012,2012-04-12 13:27:53,22272,2012-04-03 17:59:08,12689
4,208842,4b4606f2f964a520751426e3,Fri Apr 13 15:41:49 +0000 2012,-300,Apr 13 15:41:49 2012,2012-04-13 15:41:49,22272,2012-04-03 17:59:08,14263


In [14]:
# Sort the data before storing
df_selected = df_selected.sort_values(["user","time_in_min"]).reset_index(drop=True)
df_selected.head()

,user,locid,utc,time,utc_clipped,utc_dt,locid_num,date_init,time_in_min
0,15,4b220ceef964a5206d4324e3,Fri Apr 27 19:42:16 +0000 2012,-420,Apr 27 19:42:16 2012,2012-04-27 19:42:16,19503,2012-04-03 17:59:08,34663
1,15,4f24a5f0e4b050258b801966,Sat Apr 28 07:25:03 +0000 2012,-420,Apr 28 07:25:03 2012,2012-04-28 07:25:03,58748,2012-04-03 17:59:08,35366
2,15,4f9043f2e4b06b50af9c407c,Sat Apr 28 20:03:24 +0000 2012,-420,Apr 28 20:03:24 2012,2012-04-28 20:03:24,61325,2012-04-03 17:59:08,36124
3,15,4f24a5f0e4b050258b801966,Sat Apr 28 20:03:47 +0000 2012,-420,Apr 28 20:03:47 2012,2012-04-28 20:03:47,58748,2012-04-03 17:59:08,36125
4,15,4be6f1b9cf200f47a809153c,Sat Apr 28 22:35:59 +0000 2012,-420,Apr 28 22:35:59 2012,2012-04-28 22:35:59,36967,2012-04-03 17:59:08,36277


In [15]:
# Reassign sequential user ids for the dataset (algo requirement)
users = df_selected.groupby('user', as_index = False).agg({"locid_num":"count"}).reset_index()
users.columns = ['new_userid','user','cnt']
users['new_userid'] += 1

print(users.shape)
print(users.new_userid.min(), users.new_userid.max())

users = users[['new_userid','user']]

# # Save the locid to locid_num mapping
# if small:
#   users.to_csv('Foursquare' + "/small_small_user_mapping.csv", index = False)
# else: 
#   users.to_csv(dname + "/user_mapping.csv", index = False)
users.head(2)
# (114306, 3)

# ### SMALL
# (12800, 3)
# 1 12800
# 1813
# approx 3k

(21437, 3)
1 21437


,new_userid,user
0,1,15
1,2,19


In [16]:
# Add the new user ids - rows should be same as before
print(df_selected.shape)
df_selected = df_selected.merge(users, on=['user'], how = 'inner')
print(df_selected.shape)

df_selected.head()

# (15397922,10)
# ### SMALL
# (58986, 10)

(1871034, 9)
(1871034, 10)


,user,locid,utc,time,utc_clipped,utc_dt,locid_num,date_init,time_in_min,new_userid
0,15,4b220ceef964a5206d4324e3,Fri Apr 27 19:42:16 +0000 2012,-420,Apr 27 19:42:16 2012,2012-04-27 19:42:16,19503,2012-04-03 17:59:08,34663,1
1,15,4f24a5f0e4b050258b801966,Sat Apr 28 07:25:03 +0000 2012,-420,Apr 28 07:25:03 2012,2012-04-28 07:25:03,58748,2012-04-03 17:59:08,35366,1
2,15,4f9043f2e4b06b50af9c407c,Sat Apr 28 20:03:24 +0000 2012,-420,Apr 28 20:03:24 2012,2012-04-28 20:03:24,61325,2012-04-03 17:59:08,36124,1
3,15,4f24a5f0e4b050258b801966,Sat Apr 28 20:03:47 +0000 2012,-420,Apr 28 20:03:47 2012,2012-04-28 20:03:47,58748,2012-04-03 17:59:08,36125,1
4,15,4be6f1b9cf200f47a809153c,Sat Apr 28 22:35:59 +0000 2012,-420,Apr 28 22:35:59 2012,2012-04-28 22:35:59,36967,2012-04-03 17:59:08,36277,1


In [17]:
# Drop the additional columns
df_final = df_selected[['new_userid','locid_num','time_in_min']]
print(df_final.shape)
df_final.head()
# (15397922, 3)
# ### SMALL
# (58986, 3)

(1871034, 3)


,new_userid,locid_num,time_in_min
0,1,19503,34663
1,1,58748,35366
2,1,61325,36124
3,1,58748,36125
4,1,36967,36277


In [18]:
# Convert to numpy and save as .npy file
np_final = df_final.to_numpy()
if small:
  np.save('./data/'+dname+'_small.npy', np_final)
else:
  np.save('./data/'+dname+'.npy', np_final)

### POI File Generation

In [19]:
# Load the data, remove the additional columns
df_poi = pd.read_csv('data/Foursquare/dataset_WWW2019/raw_POIs.txt', sep='\t', header=None)
df_poi.columns = ['locid','lat','long','cat','country']
df_poi = df_poi[['locid','lat','long']]
print(df_poi.shape)
df_poi.head()
# (11180160, 3)

(11180160, 3)


,locid,lat,long
0,3fd66200f964a52000e61ee3,40.729209,-73.998753
1,3fd66200f964a52000e71ee3,40.733596,-74.003139
2,3fd66200f964a52000e81ee3,40.758102,-73.975734
3,3fd66200f964a52000ea1ee3,40.732456,-74.003755
4,3fd66200f964a52000ec1ee3,42.345907,-71.087001


In [20]:
# # Load the checkins data
# if small:
#   checkins = pd.read_csv(dname+'/small_location_mapping.csv')
# else:
#   checkins = pd.read_csv(dname+'/location_mapping.csv')
# checkins = checkins[['locid_num','locid']]
# print(checkins.shape)
# checkins.head()

In [21]:
# Map the numeric location ids and sort according to them
df_poi_selected = df_poi.merge(checkins,  on=['locid'], how = 'inner')
print(df_poi_selected.shape) #(409428, 4)
df_poi_selected = df_poi_selected.sort_values('locid_num').reset_index(drop = True)[['locid_num','lat','long']]

df_poi_selected.head(2)
# (409428, 4)

(66255, 4)


,locid_num,lat,long
0,1,40.733596,-74.003139
1,2,40.758102,-73.975734


In [22]:
# Convert to numpy and save as .npy file
np_poi_final = df_poi_selected.to_numpy()
if small:
  np.save('./data/'+dname+'_small_POI.npy', np_poi_final)
else:
  np.save('./data/'+dname+'_POI.npy', np_poi_final)

## Gowalla Files

In [ ]:
# change small to False for Foursquare or to True for Foursquare_small
dname = 'Gowalla'
small = True
# min_period = dt.datetime(2010, 8, 1, 0, 0, 0)
# max_period = dt.datetime(2010, 10, 31, 23, 59, 59)

# # 5 states - east coast
# min_lat = 37.431600
# min_long = -86.134900

# max_lat = 41.203300
# max_long = -77.194500

# west coast
min_lat = 34.048900
min_long = -120.554200
max_lat = 47.751100
max_long = -105.782100

# print(min_period, max_period)
print(min_lat, max_lat, min_long, max_long)

34.0489 47.7511 -120.5542 -105.7821


In [ ]:
max_long > min_long

### Main Generation

In [ ]:
df = pd.read_csv('data/Gowalla/loc-gowalla_totalCheckins.txt', sep='\t', header=None)
df.columns = ['user','utc','lat','longi','locid']
print(df.shape) 
df.head()
# 6442892, 5)

(6442892, 5)


,user,utc,lat,longi,locid
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [ ]:
# Get date in datetime format and also get the minimum date
df['utc_dt'] = pd.to_datetime(df.utc, format = '%Y-%m-%dT%H:%M:%SZ')

# Filter dataset if small needed
if small:
  # df = df[(df.utc_dt >= min_period)&(df.utc_dt <= max_period)]
  df = df[((df.lat >= min_lat) & (df.lat <= max_lat))]
  df = df[((df.longi >= min_long) & (df.longi <= max_long))]

print(df.shape)
min_date = df.utc_dt.min() - dt.timedelta(minutes = 1)
print(min_date)

df.head(2)
# (6442892, 6)
# 2009-02-04 05:16:38

(234315, 6)
2009-03-21 04:50:18


,user,utc,lat,longi,locid,utc_dt
136,0,2010-07-25T17:19:54Z,39.191054,-106.821249,1229499,2010-07-25 17:19:54
137,0,2010-07-25T04:40:18Z,39.186190,-106.816869,166829,2010-07-25 04:40:18


In [ ]:
# Filter users with more than 5 check-ins
users = df.groupby('user',as_index=False).agg({'locid':'count'})
df_selected = df[df.user.isin(users[users.locid > 5].user)]
print(df_selected.shape)
# 6383468

(222234, 6)


In [ ]:
# Get locations with at least 10 check-ins - had same location id for different long,lat combination, thus creating locid_num for unique combn of locid, lat, long
checkins = df_selected.groupby(['locid','lat','longi'], as_index= False).agg({'user':'count'})
checkins = checkins[checkins.user > 9]
print(checkins.shape)

# Introduce the numeric location ids
checkins = checkins.reset_index(drop = True).drop(columns = ['user']).reset_index()
checkins.columns = ['locid_num', 'locid','lat','longi']
checkins['locid_num'] = checkins.locid_num + 1
print(checkins.shape)
print(checkins.locid_num.min(), checkins.locid_num.max())

# # Save the locid to locid_num mapping
# if small:
#   checkins.to_csv("Gowalla_small" + "/small_location_mapping.csv", index = False)
# else:
#   checkins.to_csv(dname + "/location_mapping.csv", index = False)
checkins.head(2)
# (130136, 4)

(4592, 4)
(4592, 4)
1 4592


,locid_num,locid,lat,longi
0,1,9413,46.918927,-114.084284
1,2,9477,34.090844,-118.370454


In [ ]:
# Filter for locations with atleast 10 check-ins and add the numeric location ids
print(df_selected.shape)
df_selected = df_selected.merge(checkins, on=['locid','lat','longi'], how = 'inner')
print(df_selected.shape)

df_selected.head(1)
# (6383468, 6)
# (3501989, 7)

(222234, 6)
(110680, 7)


,user,utc,lat,longi,locid,utc_dt,locid_num
0,0,2010-07-25T04:40:18Z,39.18619,-106.816869,166829,2010-07-25 04:40:18,1457


In [ ]:
# Filter users with more than 2 check-ins
users = df_selected.groupby('user',as_index = False).agg({'locid':'count'})
df_selected = df_selected[df_selected.user.isin(users[users.locid > 2].user)]
print(df_selected.shape)

print(df_selected.locid_num.nunique())
print(df_selected.locid_num.min(), df_selected.locid_num.max())

df_selected.head()

(109758, 7)
4592
1 4592


,user,utc,lat,longi,locid,utc_dt,locid_num
0,0,2010-07-25T04:40:18Z,39.18619,-106.816869,166829,2010-07-25 04:40:18,1457
1,91,2010-08-22T06:27:51Z,39.18619,-106.816869,166829,2010-08-22 06:27:51,1457
2,501,2010-07-25T04:59:30Z,39.18619,-106.816869,166829,2010-07-25 04:59:30,1457
3,1598,2010-10-16T04:48:01Z,39.18619,-106.816869,166829,2010-10-16 04:48:01,1457
4,1598,2010-10-09T04:13:32Z,39.18619,-106.816869,166829,2010-10-09 04:13:32,1457


In [ ]:
# converting date to time in minutes
df_selected['date_init'] = pd.to_datetime(min_date)
df_selected['time_in_min'] = (df_selected.utc_dt - df_selected.date_init).dt.total_seconds().div(60).astype(float)
df_selected.head()

,user,utc,lat,longi,locid,utc_dt,locid_num,date_init,time_in_min
0,0,2010-07-25T04:40:18Z,39.18619,-106.816869,166829,2010-07-25 04:40:18,1457,2009-03-21 04:50:18,707030.000000
1,91,2010-08-22T06:27:51Z,39.18619,-106.816869,166829,2010-08-22 06:27:51,1457,2009-03-21 04:50:18,747457.550000
2,501,2010-07-25T04:59:30Z,39.18619,-106.816869,166829,2010-07-25 04:59:30,1457,2009-03-21 04:50:18,707049.200000
3,1598,2010-10-16T04:48:01Z,39.18619,-106.816869,166829,2010-10-16 04:48:01,1457,2009-03-21 04:50:18,826557.716667
4,1598,2010-10-09T04:13:32Z,39.18619,-106.816869,166829,2010-10-09 04:13:32,1457,2009-03-21 04:50:18,816443.233333


In [ ]:
# Round the time_in_min
df_selected['time_in_min'] = df_selected['time_in_min'].round(0).astype(int)
print(df_selected.time_in_min.min(), df_selected.time_in_min.max())
df_selected.head()
# 1 901445

1 836455


,user,utc,lat,longi,locid,utc_dt,locid_num,date_init,time_in_min
0,0,2010-07-25T04:40:18Z,39.18619,-106.816869,166829,2010-07-25 04:40:18,1457,2009-03-21 04:50:18,707030
1,91,2010-08-22T06:27:51Z,39.18619,-106.816869,166829,2010-08-22 06:27:51,1457,2009-03-21 04:50:18,747458
2,501,2010-07-25T04:59:30Z,39.18619,-106.816869,166829,2010-07-25 04:59:30,1457,2009-03-21 04:50:18,707049
3,1598,2010-10-16T04:48:01Z,39.18619,-106.816869,166829,2010-10-16 04:48:01,1457,2009-03-21 04:50:18,826558
4,1598,2010-10-09T04:13:32Z,39.18619,-106.816869,166829,2010-10-09 04:13:32,1457,2009-03-21 04:50:18,816443


In [ ]:
# Sort the data before storing
df_selected = df_selected.sort_values(["user","time_in_min"]).reset_index(drop=True)
print(df_selected.shape)
df_selected.head()
# (3501989, 9))

(109758, 9)


,user,utc,lat,longi,locid,utc_dt,locid_num,date_init,time_in_min
0,0,2010-07-22T01:40:44Z,39.219111,-106.864015,54760,2010-07-22 01:40:44,532,2009-03-21 04:50:18,702530
1,0,2010-07-22T18:22:41Z,39.201128,-106.831248,406978,2010-07-22 18:22:41,2940,2009-03-21 04:50:18,703532
2,0,2010-07-25T04:40:18Z,39.186190,-106.816869,166829,2010-07-25 04:40:18,1457,2009-03-21 04:50:18,707030
3,2,2010-08-25T16:45:31Z,34.076312,-118.257073,50872,2010-08-25 16:45:31,493,2009-03-21 04:50:18,752395
4,2,2010-08-26T02:04:12Z,34.049074,-118.255905,214598,2010-08-26 02:04:12,1781,2009-03-21 04:50:18,752954


In [ ]:
# Reassign sequential user ids for the dataset (algo requirement)
users = df_selected.groupby('user', as_index = False).agg({"locid_num":"count"}).reset_index()
users.columns = ['new_userid','user','cnt']
users['new_userid'] += 1

print(users.shape)
print(users.new_userid.min(), users.new_userid.max())

users = users[['new_userid','user']]

# Save the locid to locid_num mapping
# if small:
#   users.to_csv("Gowalla_small" + "/small_user_mapping.csv", index = False)
# else: 
#   users.to_csv(dname + "/user_mapping.csv", index = False)
users.head(2)
# (80019, 3)

(4578, 3)
1 4578


,new_userid,user
0,1,0
1,2,2


In [ ]:
# Add the new user ids - rows should be same as before
print(df_selected.shape)
df_selected = df_selected.merge(users, on=['user'], how = 'inner')
print(df_selected.shape)

df_selected.head()
# (3501989, 10)

(109758, 9)
(109758, 10)


,user,utc,lat,longi,locid,utc_dt,locid_num,date_init,time_in_min,new_userid
0,0,2010-07-22T01:40:44Z,39.219111,-106.864015,54760,2010-07-22 01:40:44,532,2009-03-21 04:50:18,702530,1
1,0,2010-07-22T18:22:41Z,39.201128,-106.831248,406978,2010-07-22 18:22:41,2940,2009-03-21 04:50:18,703532,1
2,0,2010-07-25T04:40:18Z,39.186190,-106.816869,166829,2010-07-25 04:40:18,1457,2009-03-21 04:50:18,707030,1
3,2,2010-08-25T16:45:31Z,34.076312,-118.257073,50872,2010-08-25 16:45:31,493,2009-03-21 04:50:18,752395,2
4,2,2010-08-26T02:04:12Z,34.049074,-118.255905,214598,2010-08-26 02:04:12,1781,2009-03-21 04:50:18,752954,2


In [ ]:
# Drop the additional columns
df_final = df_selected[['new_userid','locid_num','time_in_min']]
print(df_final.shape)
df_final.head()
# (3501989, 3)

(109758, 3)


,new_userid,locid_num,time_in_min
0,1,532,702530
1,1,2940,703532
2,1,1457,707030
3,2,493,752395
4,2,1781,752954


In [ ]:
# Convert to numpy and save as .npy file
np_final = df_final.to_numpy()
if small:
  np.save('./data/'+dname+'_small.npy', np_final)
else:
  np.save('./data/'+dname+'.npy', np_final)

### POI File Generation

In [ ]:
df_poi_selected = df_selected[['locid_num','lat','longi']].drop_duplicates()
print(df_poi_selected.shape)
print(df_poi_selected.locid_num.nunique())
print(df_poi_selected.locid_num.min(),df_poi_selected.locid_num.max())
df_poi_selected.head()
# (130136, 3)

(4592, 3)
4592
1 4592


,locid_num,lat,longi
0,532,39.219111,-106.864015
1,2940,39.201128,-106.831248
2,1457,39.186190,-106.816869
3,493,34.076312,-118.257073
4,1781,34.049074,-118.255905


In [ ]:
# Convert to numpy and save as .npy file
np_poi_final = df_poi_selected.to_numpy()
if small:
  np.save('./data/'+dname+'_small_POI.npy', np_poi_final)
else:
  np.save('./data/'+dname+'_POI.npy', np_poi_final)